In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


llms = [ 
    "gemma3n",  
    "qwen2.5",  
    "llama3.1",  
    "orca2",  
]


examples = {
    "FIC" : "join_examples_dict_1", 
    "CIC" : 'join_examples_dict_2',
    "FSC" : 'vector_based_examples_dict_1',
    "CSC" : 'vector_based_examples_dict_2',
}

model_types = ['TF', 'FT', 'UN', 'IN']


def extract_model_type(model, suffix=""): 
    flag = True
    for ll in llms:
        if ll in model:
            flag = False
    if flag: 
        return "P1"
    elif ' U ' in model: 
        return f"UN{suffix}"
    elif ' ∩ ' in model: 
        return f"IN{suffix}"
    elif '-tf-p2' in model: 
        return "TF"
    elif '-ft-p2' in model: 
        return "FT"
    elif '-z-p2' in model: 
        return "ZS" 
    else: 
        return "P1"


# def df_plot_show(groups, datasets): 
#     prompt_types = list(groups.keys())
#     average_f1 = np.array(list(groups.values()))

#     bar_width = 0.2                   # width of each bar
#     group_spacing = 0.5  # extra space between groups

#     group_width = len(datasets) * bar_width + group_spacing
#     x = np.arange(len(prompt_types)) * group_width

#     # Create the bar plot
#     fig, ax = plt.subplots(figsize=(14, 6))
#     for i, dataset in enumerate(datasets):
#         ax.bar(x + i*bar_width, average_f1[:, i], width=bar_width, label=dataset)
        
#     # Labels and styling
#     ax.set_xlabel("Prompt Type", fontsize=12)
#     ax.set_ylabel("F1 Score", fontsize=12)
#     ax.set_title("Average F1 Score per Prompt Type and Dataset (Standard Blocking)", fontsize=14)
#     ax.set_xticks(x + bar_width*(len(datasets)-1)/2)
#     ax.set_xticklabels(prompt_types)
#     ax.legend(title="Datasets")

    
#     plt.tight_layout()

#     plt.show()
    




In [3]:
'FIC-FT'.split('-')

['FIC', 'FT']

In [19]:
prompt_types = ["ZS", "FIC-TF", "CIC-TF", "FSC-TF",
    "CSC-TF", "FIC-FT", "CIC-FT", "FSC-FT", "CSC-FT",
    "FIC-UN", "CIC-UN", "FSC-UN", "CSC-UN", "FIC-IN",
    "CIC-IN", "FSC-IN", "CSC-IN"]



def results_extraction_main(): 
    
    groups = dict()        
    datasets = ['D2', 'D5', 'D6', 'D7', 'D8']
    for candidate_pairs in ['original', 'standard_blocking']:
        for dataset in datasets:
            key = (candidate_pairs, dataset)
            groups[key] = list()
            
            results = f'../results/{candidate_pairs}/{dataset}_clustering.csv'
            results = pd.read_csv(results)

            results['model_type'] = results['model'].apply(lambda x: extract_model_type(x, ''))

            if candidate_pairs == 'original': 
                results = results[(results['model_type'] != 'P1')] 
            else: 
                results_ui = f'../results/{candidate_pairs}/{dataset}_ui.csv'
                results_ui = pd.read_csv(results_ui)
                results_ui['model_type'] = results_ui['model'].apply(lambda x: extract_model_type(x, ""))
                results = results[(results['model_type'] != 'P1') & (results['model_type'] != 'UN') & (results['model_type'] != 'IN')]  
                result_ui = results_ui[(results_ui['model_type'] != 'P1')]
                results = pd.concat([results, result_ui])  


            groupby = results.groupby(by=['model_type', 'examples'])

            for prompt_type in prompt_types:
                
                if prompt_type == 'ZS': 
                    groups[key].append(groupby.get_group((prompt_type,"vector_based_examples_dict_1"))['time (sec)'].mean())
                else:
                    prefix, model = prompt_type.split('-')
                    groups[key].append(groupby.get_group((model, examples[prefix]))['time (sec)'].mean())
    return groups
    # groups
    # df_plot_show(groups, datasets)


In [20]:

# candidate_pairs = 'standard_blocking'
groups = results_extraction_main()
# Create a MultiIndex dataframe
df = pd.DataFrame(groups, index=prompt_types)

# Style function to bold maximum values in each column
def highlight_max(s):
    is_max = s == s.min()
    return ["font-weight: bold" if v else "" for v in is_max]

styled_df = df.style.apply(highlight_max, axis=0).set_table_styles(
    {
        ("kNN-Join", "D2"): [{"selector": "th", "props": "border: 1px solid black;"}],
        ("blocking workflows", "D2"): [{"selector": "th", "props": "border: 1px solid black;"}],
    },
    overwrite=False,
)

styled_df.to_excel("runtime.xlsx", engine='openpyxl')
styled_df